<b>Star Galaxy Separation for DP0 catalogs </b> <br>
Last verified to run on 2021-08-16 with LSST Science Pipelines release w_2021_33 <br>
Authors: Leanne Guy <br>
Target audience: All DP0 delegates. <br>
Container Size: medium <br>

In [1]:
# Import general python packages
import numpy as np
import re
import pandas
from pandas.testing import assert_frame_equal

# LSST Science Pipelines 
from lsst.daf.butler import Butler
from lsst.geom import Box2I, Point2I, Extent2I, Box2D

# Rubin TAP service utilities
from rubin_jupyter_utils.lab.notebook import get_tap_service, retrieve_query

# Astropy
from astropy import units as u
from astropy.coordinates import SkyCoord

# Bokeh for interactive visualization
import bokeh
from bokeh.io import output_file, output_notebook, show
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource, CDSView, GroupFilter, HoverTool
from bokeh.plotting import figure
from bokeh.transform import factor_cmap

Patching auth into notebook.base.handlers.IPythonHandler(notebook.base.handlers.AuthenticatedHandler) -> IPythonHandler(jupyterhub.singleuser.mixins.HubAuthenticatedHandler, notebook.base.handlers.AuthenticatedHandler)


In [2]:
# Create a TAP service 
service = get_tap_service()
assert service is not None

In [3]:
# Instantiate a butler and point to DP0
repo = 's3://butler-us-central1-dp01'
collection = '2.2i/runs/DP0.1'
butler = Butler(repo, collections=collection)
assert butler is not None

Found credentials in shared credentials file: /home/leannep/.lsst/aws-credentials.ini


First calculate the ratio of the model flux to the PSF flux. This provides a measure of how much a source appears like a point source. The smaller the ratio, the more likely a source is to be a point source. 

In [42]:
import sqlparse 
fd = open('queries/dp0-1/query-star-galaxy-separation.txt', 'r')
query = fd.read()
q = sqlparse.format(query, strip_comments=True).strip()
print(q)

SELECT objectId,coord_ra,coord_dec,g_base_ClassificationExtendedness_flag
    ,g_base_ClassificationExtendedness_value,g_base_PsfFlux_instFlux,g_base_PsfFlux_instFluxErr
    ,g_good,g_modelfit_CModel_instFlux,g_modelfit_CModel_instFluxErr
FROM dp01_dc2_catalogs.forced_photometry


In [49]:
job = service.submit_job(q)

In [50]:
job.run()

In [51]:
job.wait(phases=['COMPLETED', 'ERROR'])
print('Job phase is', job.phase)

Job phase is ERROR


In [30]:
results = job.fetch_result().to_table()

AttributeError: 'NoneType' object has no attribute 'href'

In [38]:
job.delete()

AttributeError: 'NoneType' object has no attribute 'startswith'

In [37]:
# Use the Butler
rSources = butler.get('deepCoadd_forced_src', {'filter': 'HSC-R', 'tract': 0, 'patch': '1,1'})


Type mismatch found for value ''1,1'' provided for dimension patch. Could not find matching alternative (primary key has type <class 'int'>) so attempting to use as-is.


KeyError: "No value in data ID ({'filter': 'HSC-R', 'tract': 0, 'patch': '1,1'}) for required dimension 'band'."